In [1]:
from datasets import load_dataset
from dotenv import dotenv_values
from config.finetuning import preprocessing_params
from utils.data_preprocessor import DataPreprocessor

HF_TOKEN = dotenv_values(".env.base")['HF_TOKEN']
hf_e3c = load_dataset("ferrazzipietro/e3c-sentences", token = HF_TOKEN)

/Users/pietroferrazzi/Desktop/dottorato/mistral_finetuning/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [21]:
data_preprocessor = DataPreprocessor("mistralai/Mistral-7B-Instruct-v0.2", "mistralai/Mistral-7B-Instruct-v0.2", clen=True)
d = data_preprocessor.preprocess_data_one_layer(hf_e3c['en.layer1'], simplest_prompt=True)
d[10]

MODEL TYPE: mistral


{'sentence': 'Clinical examination revealed a multinodular goiter.',
 'entities': [{'id': '5611',
   'offsets': [32, 51],
   'role': '',
   'semantic_type_id': 'C0342208',
   'text': 'multinodular goiter',
   'type': 'CLINENTITY'}],
 'original_text': 'We present a case of a 32-year-old woman with a history of gradual enlargement of the anterior neck. The medical history was unremarkable and no comorbidities existed. There was no history of radiation exposure. Clinical examination revealed a multinodular goiter.\r\nComputed tomography showed a thyroid nodule of the left lobe extended to the isthmus and the right lobe with anterior and posterior capsular rupture contracting close contact with the vascular axis left carotid-jugular plunging into the cervicothoracic away from the hole aortic arch.\r\nThere was no evidence of lung lesions. The patient underwent a surgical exploration. There was a hard left lobe nodule of 5 cm infiltrating the adjacent muscles and partially infiltrates the t

In [8]:
count_clin = 0
count_other = 0

for ent_list in hf_e3c['en.layer1']['entities']:
    for ent in ent_list:
        if ent['type'] == 'CLINENTITY':
            count_clin = count_clin + 1
        else:
            count_other = count_other + 1
count_clin, count_other

(975, 7392)

In [13]:
hf_e3c['en.layer1'][159]

{'sentence': 'We present a case of a 77-year-old Caucasian woman with obstructive jaundice.',
 'entities': [{'id': '1567',
   'offsets': [3, 10],
   'role': '',
   'semantic_type_id': '',
   'text': 'present',
   'type': 'EVENT'},
  {'id': '1582',
   'offsets': [68, 76],
   'role': '',
   'semantic_type_id': '',
   'text': 'jaundice',
   'type': 'EVENT'},
  {'id': '1882',
   'offsets': [56, 76],
   'role': '',
   'semantic_type_id': 'C0022354',
   'text': 'obstructive jaundice',
   'type': 'CLINENTITY'},
  {'id': '1948',
   'offsets': [21, 50],
   'role': 'PATIENT',
   'semantic_type_id': '',
   'text': 'a 77-year-old Caucasian woman',
   'type': 'ACTOR'}],
 'original_text': "We present a case of a 77-year-old Caucasian woman with obstructive jaundice. Laboratory tests revealed that she had elevated bilirubin and liver enzyme levels. A computed tomographic scan showed a homogeneous 5 × 3-cm mass obstructing the common bile duct. The results of brush cytology were consistent with a bile

In [ ]:
data_preprocessor = DataPreprocessor("mistralai/Mistral-7B-Instruct-v0.2", "mistralai/Mistral-7B-Instruct-v0.2")
d = data_preprocessor.preprocess_data_one_layer(hf_e3c['en.layer1'], simplest_prompt=True)
d['prompt']
# splits = ['en.layer1', 'en.layer2', 'en.layer2.validation', 'en.layer3',
#           'es.layer1', 'es.layer2', 'es.layer2.validation', 'es.layer3',
#           'eu.layer1', 'eu.layer2', 'eu.layer2.validation', 'eu.layer3',
#           'it.layer1', 'it.layer2', 'it.layer2.validation', 'it.layer3',
#           'fr.layer1', 'fr.layer2', 'fr.layer2.validation', 'fr.layer3']
# for split_name in splits:
#     hf_e3c[split_name] = data_preprocessor.apply(data=hf_e3c[split_name], 
#                                                 task=preprocessing_params.task, 
#                                                 instruction_on_response_format=preprocessing_params.instruction_on_response_format, 
#                                                 n_shots=preprocessing_params.n_shots, 
#                                                 offset=preprocessing_params.offset, 
#                                                 tokenizer=preprocessing_params.tokenizer, 
#                                                 list_of_examples=preprocessing_params.list_of_examples,
#                                                 list_of_responses=preprocessing_params.list_of_responses)

In [ ]:
import pandas as pd
d = 'AAA_4bit_base.csv'
df0 = pd.read_csv(d)
df0['f1_score'].hist()
df0['f1_score'].max()

In [ ]:
import pandas as pd
d = 'AAA_4bit_base_None.csv'
df0 = pd.read_csv(d)
df0['f1_score'].hist()
df0['f1_score'].max()

In [ ]:
import pandas as pd
d = "AAA_7B_NoQuant_FT_float16.csv"
df = pd.read_csv(d)
# l1 = df['file'].to_list()
# l2 = df0['file'].to_list()
# idx = [i for i in l1 if i in l2]
# out = df[df['file'].isin(idx)]
df['f1_score'].hist()

In [ ]:
model_id.split('/')[1].lower().split('-')[0]
'lama3' in model_id

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from dotenv import dotenv_values

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
HF_TOKEN = dotenv_values(".env.base")['HF_TOKEN']

tokenizer = AutoTokenizer.from_pretrained(model_id,
                                          token = HF_TOKEN,
                                          cache_dir='/data/disk1/share/pferrazzi/.cache')

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    load_in_4bit=True,
    token = HF_TOKEN,
    cache_dir='/data/disk1/share/pferrazzi/.cache'
)

messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"},
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]


# outputs = model.generate(
#     input_ids,
#     max_new_tokens=256,
#     eos_token_id=terminators,
#     do_sample=True,
#     temperature=0.6,
#     top_p=0.9,
# )
# response = outputs[0][input_ids.shape[-1]:]
# print(tokenizer.decode(response, skip_special_tokens=True))


In [ ]:
input_ids

In [ ]:
tokenizer.decode(input_ids[0])

In [ ]:
from dotenv import dotenv_values
from datasets import load_dataset, Dataset
from utils.data_preprocessor import Slovenian_preprocessor
from utils.test_data_processor import TestDataProcessSlovenian as TestDataProcessor
from utils.generate_ft_adapters_list import generate_ft_adapters_list
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
import gc
from peft import PeftModel
from tqdm import tqdm
import pandas as pd

from config import postprocessing_params_llama as postprocessing
from log import slo_llama7B_NoQuant as models_params
adapters_list = generate_ft_adapters_list("slo_llama7B_NoQuant", simplest_prompt=models_params.simplest_prompt)
print(adapters_list)
HF_TOKEN = dotenv_values(".env.base")['HF_TOKEN']
LLAMA_TOKEN = dotenv_values(".env.base")['LLAMA_TOKEN']

max_new_tokens_factor_list = postprocessing.max_new_tokens_factor_list
n_shots_inference_list = postprocessing.n_shots_inference_list
layer = models_params.TRAIN_LAYER
language = 'slo'

tokenizer = AutoTokenizer.from_pretrained(models_params.BASE_MODEL_CHECKPOINT, add_eos_token=False,
                                         token=LLAMA_TOKEN)


val_data = pd.read_csv(models_params.slovenian_test_path, header=None, names=['word', 'label'])
preprocessor = Slovenian_preprocessor(val_data, models_params.BASE_MODEL_CHECKPOINT, tokenizer, token_llama=HF_TOKEN)
preprocessor.preprocess()
preprocessor.apply('', offset=False, simplest_prompt=False)
preprocessor.extract_sentence_from_prompt_col()
val_data = preprocessor.data
val_data = val_data.shuffle(seed=1234)  # Shuffle dataset here
val_data = val_data.map(lambda samples: tokenizer(samples[models_params.dataset_text_field]), batched=True)

      

In [ ]:
base_model

In [ ]:
bnb_config = BitsAndBytesConfig(
                            load_in_4bit=True,
                            )
base_model = AutoModelForCausalLM.from_pretrained(
                    models_params.BASE_MODEL_CHECKPOINT, low_cpu_mem_usage=True,
                    quantization_config = bnb_config,
                    return_dict=True,  
                    device_map= "auto",
                    token=LLAMA_TOKEN,
                    cache_dir='/data/disk1/share/pferrazzi/.cache')

In [ ]:

adapters ='ferrazzipietro/Llama-2-7b-chat-hf_adapters_SLO_NoQuant_torch.bfloat16_32_32_0.01_1_0.0002'
merged_model = PeftModel.from_pretrained(base_model, 
                                                    adapters, 
                                                    token=HF_TOKEN, 
                                                    device_map='auto',
                                                    is_trainable = False)
tokenizer = AutoTokenizer.from_pretrained(models_params.BASE_MODEL_CHECKPOINT, 
                                                    add_eos_token=False,
                                                    token=LLAMA_TOKEN)
tokenizer.pad_token = tokenizer.eos_token# "<pad>" #tokenizer.eos_token
tokenizer.padding_side = "left"
postprocessor = TestDataProcessor(test_data=val_data.select(range(8)), 
                                            preprocessor=preprocessor, 
                                            n_shots_inference=0, 
                                            language=language, 
                                            tokenizer=tokenizer)
postprocessor.add_inference_prompt_column(simplest_prompt=False)
            # tmp = []
            # for example in postprocessor.test_data:
            #     tmp.append(example)
            # import pandas as pd
            # tmp = pd.DataFrame(tmp)
            # tmp = tmp.iloc[tmp['inference_prompt'].str.len().argsort()]
            # postprocessor.test_data = Dataset.from_pandas(tmp)
postprocessor.add_ground_truth_column()
            #try:
postprocessor.add_responses_column(model=merged_model, 
                                            tokenizer=tokenizer, 
                                            batch_size=postprocessing.batch_size, 
                                            max_new_tokens_factor=2)

In [ ]:
postprocessor.test_data[0]['inference_prompt']

In [ ]:
t = tokenizer("Having a background history of fever, backache, skin rash")
print(t)
tokenizer.convert_ids_to_tokens(t['input_ids'])

In [1]:
from utils.data_preprocessor import DataPreprocessorTag
from config.finetuning_llama2_tag import training_params, lora_params, model_loading_params, config, preprocessing_params
from dotenv import dotenv_values
from transformers import AutoTokenizer
from datasets import load_dataset
HF_TOKEN_WRITE = dotenv_values(".env.base")['HF_TOKEN_WRITE']

tokenizer = AutoTokenizer.from_pretrained(config.BASE_MODEL_CHECKPOINT, add_eos_token=False,
                                            token = HF_TOKEN_WRITE) #, cache_dir='/data/disk1/share/pferrazzi/.cache')
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'
dataset = load_dataset(config.DATASET_CHEKPOINT, split=config.TRAIN_LAYER) #download_mode="force_redownload"
# dataset = dataset[config.TRAIN_LAYER]
dataset = dataset.shuffle(seed=1234)  # Shuffle dataset here
preprocessor = DataPreprocessorTag(config.BASE_MODEL_CHECKPOINT, 
                                   tokenizer, token_llama=HF_TOKEN_WRITE, 
                                   data =dataset, 
                                   tagging_string=preprocessing_params.tagging_string)
preprocessor.apply(instruction_on_response_format=preprocessing_params.instruction_on_response_format)
dataset_processed = preprocessor.data.map(lambda samples: tokenizer(samples[training_params.dataset_text_field]), batched=True)
train_data, val_data, test_data = preprocessor.split_layer_into_train_val_test_(dataset_processed, config.TRAIN_LAYER)

/home/pferrazzi/mistral_finetuning/utils/data_preprocessor.py:553: SyntaxWarning: invalid escape sequence '\s'
  df = pd.read_csv(file_path,sep='\s+', header=None, names=['word', 'label'])
/home/pferrazzi/mistral_finetuning/.venv/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [6]:
train_data['prompt']

['<s>[INST] Extract the entities contained in the text, inserting the tag <tag> before and the the tag </tag> after each entity. <<Their child acquired walking at the age of 14 months.>>> [/INST]<tag>Their child</tag> <tag>acquired</tag> <tag>walking</tag> at the age of 14 months.</s>',
 '<s>[INST] Extract the entities contained in the text, inserting the tag <tag> before and the the tag </tag> after each entity. <<An abdominal ultrasound examination was reported as normal.>>> [/INST]An abdominal ultrasound <tag>examination</tag> was <tag>reported</tag> as <tag>normal</tag>.</s>',
 '<s>[INST] Extract the entities contained in the text, inserting the tag <tag> before and the the tag </tag> after each entity. <<Her blood pressure was 100/70 mmHg with a pulse rate of 98 beats/min, respiratory rate about 16/min and oral temperature of 37°C.>>> [/INST]Her <tag>blood</tag> <tag>pressure</tag> was <tag>100/70 mmHg</tag> with a pulse <tag>rate</tag> of <tag>98 beats/min</tag>, respiratory <tag

In [47]:
def _helper(example, entity_type_filter, entity_type):
            base_sentence = example['sentence']
            entities = example['entities']
            entities = sorted(entities, key=lambda x: x['offsets'][0])
            sentence = ''
            prev_end=0
            if entity_type_filter: entities = [ent for ent in entities if ent['type'] == entity_type]

            # Initialize an empty list to store the filtered data, removing overlapping offsets
            filtered_entities = []
            current_end = -1
            for item in entities:
                start, end = item['offsets']
                if start > current_end:
                    filtered_entities.append(item)
                    current_end = end
            while len(filtered_entities) > 0:
                ent = filtered_entities.pop(0)
                start = ent['offsets'][0]
                end = ent['offsets'][1]
                sentence = sentence + base_sentence[prev_end:start] + '<tag>' + base_sentence[start:end] + '</tag>'
                prev_end = end
                if len(filtered_entities) == 0:
                    sentence = sentence + base_sentence[end:]
                    break
            if sentence=='':
                sentence = base_sentence
            example['prompt'] = sentence
            return example

In [49]:
_helper(dataset[9], False, 'CLINENTITY')

{'sentence': 'Twenty five days later, she returned to obstetric emergencies for significant bleeding with severe anemia with hemoglobin 9 g/dL without signs of hypovolemic shock.',
 'entities': [{'id': '5250',
   'offsets': [28, 36],
   'role': '',
   'semantic_type_id': '',
   'text': 'returned',
   'type': 'EVENT'},
  {'id': '5267',
   'offsets': [78, 86],
   'role': '',
   'semantic_type_id': '',
   'text': 'bleeding',
   'type': 'EVENT'},
  {'id': '5282',
   'offsets': [99, 105],
   'role': '',
   'semantic_type_id': '',
   'text': 'anemia',
   'type': 'EVENT'},
  {'id': '5297',
   'offsets': [111, 121],
   'role': '',
   'semantic_type_id': '',
   'text': 'hemoglobin',
   'type': 'EVENT'},
  {'id': '5316',
   'offsets': [137, 142],
   'role': '',
   'semantic_type_id': '',
   'text': 'signs',
   'type': 'EVENT'},
  {'id': '5335',
   'offsets': [158, 163],
   'role': '',
   'semantic_type_id': '',
   'text': 'shock',
   'type': 'EVENT'},
  {'id': '5943',
   'offsets': [78, 86],
   

In [ ]:
from datasets import load_dataset

d0 = load_dataset("csv", data_files="data/llama/tag_7B_NoQuant_FT/maxNewTokensFactor6_nShotsInference0_llama-2-7b-chat-hf_adapters_en.layer1_NoQuant_TAG_32_64_0.01_2_0.0002.csv")


In [ ]:
d0['train'][0]

In [ ]:
d1['train'][0]

In [ ]:
d2['train'][0]